import csv  # To read and write CSV files
from torch.utils.data.dataset import random_split  # For splitting datasets
import torch  # PyTorch library for deep learning
import torch.nn as nn  # For neural network modules
import torch.optim as optim  # For optimization algorithms
from torch.utils import data  # For data utilities
from torch.utils.data import DataLoader  # For creating data loaders
import torchvision.transforms as transforms  # For data augmentation and transforms
import cv2  # For image processing with OpenCV
import numpy as np  # For numerical operations
import matplotlib.pyplot as plt  # For plotting graphs and images
import matplotlib.image as mpimg  # For image operations
import keras  # Keras deep learning library
from keras.models import Sequential  # For creating sequential models
from keras.optimizers import Adam  # For the Adam optimizer
from keras.layers import Convolution2D, Dense, MaxPooling2D, Dropout, Flatten  # For creating layers in a model
from sklearn.utils import shuffle  # For shuffling data
from sklearn.model_selection import train_test_split  # For splitting data into train and test sets
from imgaug import augmenters as iaa  # For image augmentation
import pandas as pd  # For data manipulation with DataFrames
import random  # For generating random numbers
import ntpath  # For OS path operations

In [ ]:
# Check system architecture and setup PyTorch environment
from os.path import exists
from wheel.pep425tags import get_abbr_impl, get_impl_ver, get_abi_tag
platform = '{}{}-{}'.format(get_abbr_impl(), get_impl_ver(), get_abi_tag())
cuda_output = !ldconfig -p|grep cudart.so|sed -e 's/.*\.\([0-9]*\)\.\([0-9]*\)$/cu\1\2/'
accelerator = cuda_output[0] if exists('/dev/nvidia0') else 'cpu'

# Install the appropriate PyTorch version based on whether CUDA is available
!pip install -q http://download.pytorch.org/whl/{accelerator}/torch-0.4.1-{platform}-linux_x86_64.whl torchvision

In [ ]:
# Download the dataset used for training the self-driving car model
# The dataset is a zip file containing images and driving logs
!wget https://d17h27t6h515a5.cloudfront.net/topher/2016/December/584f6edd_data/data.zip

## Getting the Data Set

In [ ]:
!wget https://d17h27t6h515a5.cloudfront.net/topher/2016/December/584f6edd_data/data.zip

In [ ]:
!unzip data.zip

## Reading and splitting the data

In [ ]:
# Step1: Read from the log file
samples = []
with open('/content/data/driving_log.csv') as csvfile:
    reader = csv.reader(csvfile)
    next(reader, None)
    for line in reader:
        samples.append(line)

# Step2: Divide the data into training set and validation set

train_len = int(0.8*len(samples))
valid_len = len(samples) - train_len

train_samples, validation_samples = data.random_split(samples, lengths=[train_len, valid_len])

In [ ]:
print('Training Samples: {}\nValid Samples: {}'.format(len(train_samples), len(validation_samples)))

Training Samples: 6428
Valid Samples: 1608


## Loading Images in Dataloader

In [ ]:
import cv2
import torch
import torch.utils.data as data
import numpy as np

# Step3a: Define the augmentation, transformation processes, parameters and dataset for dataloader
def augment(imgName, angle):
  name = 'data/IMG/' + imgName.split('/')[-1]
  current_image = cv2.imread(name)
  current_image = current_image[65:-25, :, :]
  if np.random.rand() < 0.5:
    current_image = cv2.flip(current_image, 1)
    angle = angle * -1.0
  return current_image, angle

In [ ]:

class Dataset(data.Dataset):

    def __init__(self, samples, transform=None):

        self.samples = samples
        self.transform = transform

    def __getitem__(self, index):

        batch_samples = self.samples[index]

        steering_angle = float(batch_samples[3])

        center_img, steering_angle_center = augment(batch_samples[0], steering_angle)
        left_img, steering_angle_left = augment(batch_samples[1], steering_angle + 0.4)
        right_img, steering_angle_right = augment(batch_samples[2], steering_angle - 0.4)

        center_img = self.transform(center_img)
        left_img = self.transform(left_img)
        right_img = self.transform(right_img)

        return (center_img, steering_angle_center), (left_img, steering_angle_left), (right_img, steering_angle_right)

    def __len__(self):
        return len(self.samples)


In [ ]:
params = {'batch_size': 32,
          'shuffle': True,
          'num_workers': 4}

from torch.utils.data import DataLoader
import torchvision.transforms as transforms

# Step3b: Creating generator using the dataloader to parallasize the process
transformations = transforms.Compose([transforms.Lambda(lambda x: (x / 255.0) - 0.5)])

training_set = Dataset(train_samples, transformations)
training_generator = data.DataLoader(training_set, **params)

validation_set = Dataset(validation_samples, transformations)
validation_generator = data.DataLoader(validation_set, **params)


## Model Architecture

In [ ]:
# Step4: Define the network

import torch.nn as nn
import torch.nn.init as init
import torch.optim as optim
import torch.nn.functional as F


class NetworkLight(nn.Module):

    def __init__(self):
        super(NetworkLight, self).__init__()
        self.conv_layers = nn.Sequential(
            nn.Conv2d(3, 24, 3, stride=2),
            nn.ELU(),
            nn.Conv2d(24, 48, 3, stride=2),
            nn.MaxPool2d(4, stride=4),
            nn.Dropout(p=0.25)
        )
        self.linear_layers = nn.Sequential(
            nn.Linear(in_features=48*4*19, out_features=50),
            nn.ELU(),
            nn.Linear(in_features=50, out_features=10),
            nn.Linear(in_features=10, out_features=1)
        )


    def forward(self, input):
        input = input.view(input.size(0), 3, 70, 320)
        output = self.conv_layers(input)
        print(output.shape)
        output = output.view(output.size(0), -1)
        output = self.linear_layers(output)
        return output

## Optimizer and Criterion

In [ ]:
# Step5: Define optimizer

model = NetworkLight()
optimizer = optim.Adam(model.parameters(), lr=0.0001)

criterion = nn.MSELoss()

In [ ]:
# Step6: Check the device and define function to move tensors to that device

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print('device is: ', device)

device is:  cuda


In [ ]:
def toDevice(datas, device):

  imgs, angles = datas
  return imgs.float().to(device), angles.float().to(device)

## Training

In [ ]:
max_epochs = 22

for epoch in range(max_epochs):

    model.to(device)

    # Training
    train_loss = 0
    model.train()
    for local_batch, (centers, lefts, rights) in enumerate(training_generator):
        # Transfer to GPU
        centers, lefts, rights = toDevice(centers, device), toDevice(lefts, device), toDevice(rights, device)

        # Model computations
        optimizer.zero_grad()
        datas = [centers, lefts, rights]
        for data in datas:
            imgs, angles = data
#             print("training image: ", imgs.shape)
            outputs = model(imgs)
            loss = criterion(outputs, angles.unsqueeze(1))
            loss.backward()
            optimizer.step()

            train_loss += loss.data.item()

        if local_batch % 100 == 0:
            print('Loss: %.3f '
                % (train_loss/((local_batch+1)*3)))




Streaming output truncated to the last 5000 lines.
torch.Size([32, 48, 4, 19])
torch.Size([32, 48, 4, 19])
torch.Size([32, 48, 4, 19])
torch.Size([32, 48, 4, 19])
torch.Size([32, 48, 4, 19])
torch.Size([32, 48, 4, 19])
torch.Size([32, 48, 4, 19])
torch.Size([32, 48, 4, 19])
torch.Size([32, 48, 4, 19])
torch.Size([32, 48, 4, 19])
torch.Size([32, 48, 4, 19])
torch.Size([32, 48, 4, 19])
torch.Size([32, 48, 4, 19])
torch.Size([32, 48, 4, 19])
torch.Size([32, 48, 4, 19])
torch.Size([32, 48, 4, 19])
torch.Size([32, 48, 4, 19])
torch.Size([32, 48, 4, 19])
torch.Size([32, 48, 4, 19])
torch.Size([32, 48, 4, 19])
torch.Size([32, 48, 4, 19])
torch.Size([32, 48, 4, 19])
torch.Size([32, 48, 4, 19])
torch.Size([32, 48, 4, 19])
torch.Size([32, 48, 4, 19])
torch.Size([32, 48, 4, 19])
torch.Size([32, 48, 4, 19])
torch.Size([32, 48, 4, 19])
torch.Size([32, 48, 4, 19])
torch.Size([32, 48, 4, 19])
torch.Size([32, 48, 4, 19])
torch.Size([32, 48, 4, 19])
torch.Size([32, 48, 4, 19])
torch.Size([32, 48, 4, 19

## Validation

In [ ]:
    # Validation
    model.eval()
    valid_loss = 0
    with torch.set_grad_enabled(False):
        for local_batch, (centers, lefts, rights) in enumerate(validation_generator):
            # Transfer to GPU
            centers, lefts, rights = toDevice(centers, device), toDevice(lefts, device), toDevice(rights, device)

            # Model computations
            optimizer.zero_grad()
            datas = [centers, lefts, rights]
            for data in datas:
                imgs, angles = data
#                 print("Validation image: ", imgs.shape)
                outputs = model(imgs)
                loss = criterion(outputs, angles.unsqueeze(1))

                train_loss += loss.data.item()

            avg_valid_loss = valid_loss/(local_batch+1)
            if local_batch % 100 == 0:
                print('Valid Loss: %.3f '
                    % (valid_loss/(local_batch+1)))

torch.Size([32, 48, 4, 19])
torch.Size([32, 48, 4, 19])
torch.Size([32, 48, 4, 19])
Valid Loss: 0.000 
torch.Size([32, 48, 4, 19])
torch.Size([32, 48, 4, 19])
torch.Size([32, 48, 4, 19])
torch.Size([32, 48, 4, 19])
torch.Size([32, 48, 4, 19])
torch.Size([32, 48, 4, 19])
torch.Size([32, 48, 4, 19])
torch.Size([32, 48, 4, 19])
torch.Size([32, 48, 4, 19])
torch.Size([32, 48, 4, 19])
torch.Size([32, 48, 4, 19])
torch.Size([32, 48, 4, 19])
torch.Size([32, 48, 4, 19])
torch.Size([32, 48, 4, 19])
torch.Size([32, 48, 4, 19])
torch.Size([32, 48, 4, 19])
torch.Size([32, 48, 4, 19])
torch.Size([32, 48, 4, 19])
torch.Size([32, 48, 4, 19])
torch.Size([32, 48, 4, 19])
torch.Size([32, 48, 4, 19])
torch.Size([32, 48, 4, 19])
torch.Size([32, 48, 4, 19])
torch.Size([32, 48, 4, 19])
torch.Size([32, 48, 4, 19])
torch.Size([32, 48, 4, 19])
torch.Size([32, 48, 4, 19])
torch.Size([32, 48, 4, 19])
torch.Size([32, 48, 4, 19])
torch.Size([32, 48, 4, 19])
torch.Size([32, 48, 4, 19])
torch.Size([32, 48, 4, 19])
t

## Saving the Model

In [ ]:
# Step8: Define state and save the model wrt to state

state = {
        'model': model.module if device == 'cuda' else model,
        }

torch.save(state, 'model.h5')

/usr/local/lib/python3.6/dist-packages/torch/serialization.py:402: UserWarning: Couldn't retrieve source code for container of type NetworkLight. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "
